In [22]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [23]:
def calculate_cylinder_volume(radius, height):
    """Calculate the volume of a cylinder given its radius and height."""
    import math
    return math.pi * radius**2 * height

In [ ]:
function_schema = {
    "name": "calculate_cylinder_volume",
    "description": "Calculate the volume of a cylinder given its radius and height",
    "parameters": {
        "type": "object",
        "properties": {
            "radius": {
                "type": "number",
                "description": "The radius of the cylinder base",
            },
            "height": {
                "type": "number",
                "description": "The height of the cylinder",
            },
        },
        "required": ["radius", "height"],
    },
}

In [ ]:
def chat_with_llm(user_input):
    # OpenAI API call for chat completion
    response = openai.ChatCompletion.create(
        model="gpt-4-0613",  # Function-calling supported model
        messages=[{"role": "user", "content": user_input}],
        functions=[function_schema],
        function_call="auto",  # Let the LLM decide when to use the function
    )

    # Check if a function call is needed
    if response.choices[0].message.get("function_call"):
        function_call = response.choices[0].message["function_call"]
        # Extract arguments
        arguments = eval(function_call["arguments"])  # Use safe parsing methods in production
        radius = arguments["radius"]
        height = arguments["height"]

        # Calculate volume
        volume = calculate_cylinder_volume(radius, height)

        # Send the result back to the LLM for a final response
        response = openai.ChatCompletion.create(
            model="gpt-4-0613",
            messages=[
                {"role": "user", "content": user_input},
                response.choices[0].message,  # Include the function call
                {
                    "role": "function",
                    "name": "calculate_cylinder_volume",
                    "content": str(volume),
                },
            ],
        )
    
    return response.choices[0].message["content"]

In [ ]:
# Example interaction
user_input = "What is the volume of a cylinder with radius 3 and height 5?"
output = chat_with_llm(user_input)
print(output)